In [ ]:
# import libraries
try:
  %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
#!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.15.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-01-18 20:40:29--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.005s  

2024-01-18 20:40:30 (62.9 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2024-01-18 20:40:30--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [ ]:
#Fetch the flat file
df_train = pd.read_csv(train_file_path,
                       sep = '\t',
                       header = None)

# Avoid repeated and unnecessary importations of the dataset
# from fCC's Static Assets
train_df =  df_train.copy()

In [ ]:
train_df.head()

,0,1
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
col_names = ['category','content']
train_df.columns = col_names
train_df.head()

,category,content
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
# Fetch the flat file.
# This time pass the name of the columns
df_test = pd.read_csv(test_file_path,
                       sep = '\t',
                       names = col_names)

# Avoid repeated and unnecessary importations of the dataset
# from fCC's Static Assets
test_df =  df_test.copy()

In [ ]:
# Any values to impute?
train_df.isna()\
        .value_counts()

category  content
False     False      4179
dtype: int64

In [ ]:
test_df.isna()\
        .value_counts()

category  content
False     False      1392
dtype: int64

In [ ]:
# Map the labels assigned for each message
categories_df = test_df.category\
           .astype('category')
d = dict(enumerate(categories_df.cat.categories))
print(d)

# Reverse / invert the dictionary mapping
inv_map = {v: k for k,
           v in d.items()}
inv_map

{0: 'ham', 1: 'spam'}


{'ham': 0, 'spam': 1}

In [ ]:
# Substitute the categorical values for numerical ones
train_df['category'] = train_df['category'].replace(inv_map)
test_df['category'] = test_df['category'].replace(inv_map)

test_df.head()

,category,content
0,0,i am in hospital da. . i will return home in e...
1,0,"not much, just some textin'. how bout you?"
2,0,i probably won't eat at all today. i think i'm...
3,0,don‘t give a flying monkeys wot they think and...
4,0,who are you seeing?


In [ ]:
# Create the train and test lists/tensors of values
train_vals = [train_df['content'].values,
             train_df['category'].values]

test_vals = [test_df['content'].values,
            test_df['category'].values]

In [ ]:
# Parse these tensors into datasets by slicing these former
train_ds = (tf.data.Dataset\
               .from_tensor_slices((train_vals[0],
                                    train_vals[1])))
test_ds = (tf.data.Dataset\
              .from_tensor_slices((test_vals[0],
                                   test_vals[1])))

In [ ]:
# Shuffle and batch the training dataset to avoid potential overfitting issues
train_ds = train_ds.shuffle(buffer_size = 100)
train_ds = train_ds.batch(64)

test_ds = test_ds.batch(64)

In [ ]:
# Map and adapt the text features to integer sequences
max_vocab  = 1000

vectorize_layer = keras.layers.TextVectorization(
                  max_tokens = max_vocab,
                  output_mode = 'int'
                  )

vectorize_layer.adapt(train_ds.map(lambda text, label: text))

In [ ]:
# Create the model
model = keras.Sequential([
    vectorize_layer,
    Embedding(
        input_dim = len(vectorize_layer.get_vocabulary()),
        output_dim = 64,
        mask_zero = True
            ),
    Bidirectional(LSTM(64,
                       return_sequences = True)),
    Bidirectional(LSTM(32)),
    Dense(64,
          activation = 'relu'),
    Dense(1),
    Dropout(.5)
])

model.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_18 (Tex  (None, None)              0         
 tVectorization)                                                 
                                                                 
 embedding_30 (Embedding)    (None, None, 64)          64000     
                                                                 
 bidirectional_58 (Bidirect  (None, None, 128)         66048     
 ional)                                                          
                                                                 
 bidirectional_59 (Bidirect  (None, 64)                41216     
 ional)                                                          
                                                                 
 dense_58 (Dense)            (None, 64)                4160      
                                                     

In [ ]:
# Define the logg-loss metric with BCE
bce = keras.losses.BinaryCrossentropy(from_logits = True)

# Configure the model for training
model.compile(optimizer = 'Adam',
              loss = bce,
              metrics = ['accuracy'])

# Train the model
EPOCHS = 15
val_steps = 10

history = model.fit(train_ds,
                    epochs = EPOCHS,
                    validation_data = test_ds,
                    validation_steps = val_steps)

Epoch 1/15
66/66 [==============================] - 34s 271ms/step - loss: 0.5027 - accuracy: 0.8792 - val_loss: 0.1360 - val_accuracy: 0.9359
Epoch 2/15
66/66 [==============================] - 12s 180ms/step - loss: 0.3756 - accuracy: 0.9227 - val_loss: 0.0889 - val_accuracy: 0.9781
Epoch 3/15
66/66 [==============================] - 12s 184ms/step - loss: 0.3718 - accuracy: 0.9229 - val_loss: 0.1198 - val_accuracy: 0.9781
Epoch 4/15
66/66 [==============================] - 12s 183ms/step - loss: 0.3601 - accuracy: 0.9299 - val_loss: 0.0943 - val_accuracy: 0.9812
Epoch 5/15
66/66 [==============================] - 12s 189ms/step - loss: 0.3552 - accuracy: 0.9304 - val_loss: 0.1129 - val_accuracy: 0.9766
Epoch 6/15
66/66 [==============================] - 13s 194ms/step - loss: 0.3556 - accuracy: 0.9368 - val_loss: 0.1296 - val_accuracy: 0.9781
Epoch 7/15
66/66 [==============================] - 12s 177ms/step - loss: 0.3598 - accuracy: 0.9273 - val_loss: 0.1017 - val_accuracy: 0.9781

In [ ]:
# Return loss and accuracy values
score = model.evaluate(test_ds)

print(' Test Loss: {:.3f},\n Test Accuracy: {:.3f}'\
      .format(score[0],
              score[1]))


22/22 [==============================] - 1s 44ms/step - loss: 0.0620 - accuracy: 0.9813
 Test Loss: 0.062,
 Test Accuracy: 0.981


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  message = np.array([pred_text])

  prediction = model.predict(message)[0][0]

  category = 'spam' if prediction > 0 else 'ham'

  return [prediction.round(2), category]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 13s 13s/step
[-7.45, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 39ms/step
You passed the challenge. Great job!
